<a href="https://colab.research.google.com/github/KTH-EXPECA/examples/blob/main/tecosa_demo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authentication and Dependencies

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [3]:
import os, re
from getpass import getpass

with open('/content/demo_project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [4]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.8/487.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import packages

In [5]:
import json, time
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, restart_sdr, make_sdr_ni, make_sdr_mango, sdr_tools

# Reserve resources

In the next cell, we reserve the required equipment and resources to form an end to end sdr wifi setup. We reserve 2 SDRs and 1 workers.

In [6]:
# reserve sdr-09 RJ45
sdr09_lease = show_reservation_byname("sdr-09-lease")
if not sdr09_lease:
    sdr09_lease = reserve(
        { "type":"network", "name": "sdr-09", "net_name": "sdr-09", "segment_id": "117", "duration": { "days":7, "hours":0 } }
    )

# reserve sdr-04 RJ45
sdr04_lease = show_reservation_byname("sdr-04-lease")
if not sdr04_lease:
    sdr04_lease = reserve(
        { "type":"network", "name": "sdr-04", "net_name": "sdr-04", "segment_id": "107", "duration": { "days":7, "hours":0 } }
    )

# reserve worker-04
worker10_lease = show_reservation_byname("worker-10-lease")
if not worker10_lease:
    worker10_lease = reserve(
        { "type":"device", "name":"worker-10", "duration": { "days":7, "hours":0 } }
    )
worker10_reservation_id = worker10_lease["reservations"][0]["id"]


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

[
    {
        "name": "sdr-09-lease",
        "id": "0c9e0e6e-d013-425f-a390-8e0ba8f97b4e",
        "reservation_id": "83497084-1f33-4d3c-b19b-ccb24d17207d",
        "status": "ACTIVE",
        "end_date": "2023-09-08T07:43:00.000000"
    },
    {
        "name": "sdr-04-lease",
        "id": "a1b54b97-a4f0-4b7f-be68-e98dae815019",
        "reservation_id": "82cc3ae6-9262-48bb-91e8-0c3b02024566",
        "status": "ACTIVE",
        "end_date": "2023-09-08T07:12:00.000000"
    },
    {
        "name": "worker-10-lease",
        "id": "a504e12a-8564-4b02-9c4c-fb19b46e6d7f",
        "reservation_id": "8504bcba-6b19-4344-a1ba-9c5a3f550831",
        "status": "ACTIVE",
        "end_date": "2023-09-07T16:25:00.000000"
    }
]


# Configure the SDRs

## Change the sdrs' firmwares to Mango

In [5]:
# change sdr-04 design to mango using worker-10
sdrname = "sdr-04"
sdrnet = chi.network.get_network(sdrname+"-net")
make_sdr_mango(sdrname,sdrnet['id'],worker10_reservation_id,"ens1")

# wait 10 seconds
time.sleep(10)

# change sdr-09 design to mango using worker-10
sdrname = "sdr-09"
sdrnet = chi.network.get_network(sdrname+"-net")
make_sdr_mango(sdrname,sdrnet['id'],worker10_reservation_id,"ens1")

2023-09-01 08:04:04.639 | SUCCESS  | chi.expeca:make_sdr_mango:299 - created make-sdr-mango container.
2023-09-01 08:04:04.641 | INFO     | chi.expeca:make_sdr_mango:301 - waiting 2 minutes for the sdr-09 to change design.
2023-09-01 08:05:06.410 | SUCCESS  | chi.expeca:make_sdr_mango:315 - SERVICE=change_design env variable is set
running change_design...
You have chosen sdr-09 and design mango
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': False}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13', 'up': True}}
SSHing to 10.30.1.18, username:root, password: 
running command on 10.30.1.18: 
cp /uboot/mango_bootbin/BOOT.bin /uboot/ ; /sbin/reboot > /dev/null 2>&1 &
command stdout: 

command sdterr: 

Waiting 200 seconds for the new desgin to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 design has be

## Reboot the SDRs

In [20]:
# reset SDR-04 using worker-10
sdrname = "sdr-04"
sdrnet = chi.network.get_network(sdrname+"-net")
restart_sdr(sdrname,sdrnet['id'],worker10_reservation_id,"ens1")

# wait 10 seconds
time.sleep(10)

# reset SDR-09 using worker-10
sdrname = "sdr-09"
sdrnet = chi.network.get_network(sdrname+"-net")
restart_sdr(sdrname,sdrnet['id'],worker10_reservation_id,"ens1")

2023-09-01 10:10:24.442 | SUCCESS  | chi.expeca:restart_sdr:255 - created reboot-sdr container.
2023-09-01 10:10:24.445 | INFO     | chi.expeca:restart_sdr:257 - waiting 2 minutes for the sdr-09 to reboot.
2023-09-01 10:11:15.129 | SUCCESS  | chi.expeca:restart_sdr:267 - SERVICE=reboot env variable is set
running reboot...
You have chosen sdr-09
sdr-09 is reachable: {'mango': {'ip': '10.30.1.17', 'port': '22', 'tenant-port': 'te1/0/26', 'up': True}, 'ni': {'ip': '10.30.1.18', 'port': '22', 'tenant-port': 'te4/0/13'}}
SSHing to 10.30.1.17, username:root, password: root
running command on 10.30.1.17: 
/sbin/reboot > /dev/null 2>&1 &
Waiting 200 seconds for the sdr to load
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
10 seconds to the next poll...
sdr-09 mango is up again.

2023-09-01 10:11:15.612 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for reboot-sdr container to be removed
2023-09-01 10:11:20.836 | INFO

## Run a WiFi access point on SDR-04

In [22]:
# make SDR-04 wifi access point using worker-10
sdrname = "sdr-04"
sdrnet = chi.network.get_network(sdrname+"-net")
sdr_tools(
    sdr_name = sdrname,
    sdr_net_id = sdrnet['id'],
    environment = {
        "SERVICE":"start_mango",
        "DESIGN":'mango',
        "SDR":sdrname,
        "SIDE":'ap',
        "CONFIG":"{\"mac_addr\":\"40:d8:55:04:20:19\"}",
        "JSON_PATH":"sdrs.json"
    },
    waiting_iter = 4,
    waiting_sec = 10,
    key_str = "mango is running ap with ip",
    verbose = False,
    worker_reservation_id = worker10_reservation_id,
    worker_net_interface = "ens1"
)

2023-09-01 10:13:05.741 | SUCCESS  | chi.expeca:sdr_tools:389 - created sdr-04-tools container.
2023-09-01 10:13:05.744 | INFO     | chi.expeca:sdr_tools:391 - waiting 4 times each 10 seconds for the sdr-04-tools to apply.
2023-09-01 10:13:37.995 | SUCCESS  | chi.expeca:sdr_tools:403 - sdr-04-tools was successful.
2023-09-01 10:13:38.501 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for sdr-04-tools container to be removed
2023-09-01 10:13:43.733 | INFO     | chi.expeca:wait_until_container_removed:37 - container sdr-04-tools is in Deleting state.
2023-09-01 10:13:48.956 | INFO     | chi.expeca:wait_until_container_removed:37 - container sdr-04-tools is in None state.


## Run a WiFi station on SDR-09

In [23]:
# make SDR-09 wifi access point using worker-10
sdrname = "sdr-09"
sdrnet = chi.network.get_network(sdrname+"-net")
sdr_tools(
    sdr_name = sdrname,
    sdr_net_id = sdrnet['id'],
    environment = {
        "SERVICE":"start_mango",
        "DESIGN":'mango',
        "SDR":sdrname,
        "SIDE":'sta',
        "CONFIG":"{\"mac_addr\":\"40:d8:55:04:20:12\"}",
        "JSON_PATH":"sdrs.json"
    },
    waiting_iter = 4,
    waiting_sec = 10,
    key_str = "mango is running sta with ip",
    verbose = False,
    worker_reservation_id = worker10_reservation_id,
    worker_net_interface = "ens1"
)

2023-09-01 10:14:26.744 | SUCCESS  | chi.expeca:sdr_tools:389 - created sdr-09-tools container.
2023-09-01 10:14:26.746 | INFO     | chi.expeca:sdr_tools:391 - waiting 4 times each 10 seconds for the sdr-09-tools to apply.
2023-09-01 10:14:59.266 | SUCCESS  | chi.expeca:sdr_tools:403 - sdr-09-tools was successful.
2023-09-01 10:14:59.740 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for sdr-09-tools container to be removed
2023-09-01 10:15:04.981 | INFO     | chi.expeca:wait_until_container_removed:37 - container sdr-09-tools is in Deleting state.
2023-09-01 10:15:10.202 | INFO     | chi.expeca:wait_until_container_removed:37 - container sdr-09-tools is in None state.


CAUTION: In this cell we tear down all the configurations and release the reserved resources. The project will be clean afterwards.

## Check connection bandwidth and status

check connection bandwidth using iperf3 from the STA SDR

In [9]:
# check connection bandwidth from the STA SDR
sdrname = "sdr-09"
sdrnet = chi.network.get_network(sdrname+"-net")
sdr_tools(
    sdr_name = sdrname,
    sdr_net_id = sdrnet['id'],
    environment = {
        "SERVICE":"check_mango",
        "SDR":sdrname,
        "SIDE":'sta',
        "JSON_PATH":"sdrs.json",
        "SPEED_CHECK":"192.168.11.1"
    },
    waiting_iter = 1,
    waiting_sec = 60,
    key_str = "",
    verbose = True,
    worker_reservation_id = worker10_reservation_id,
    worker_net_interface = "ens1"
)

2023-09-01 09:16:44.856 | SUCCESS  | chi.expeca:sdr_tools:389 - created sdr-09-tools container.
2023-09-01 09:16:44.857 | INFO     | chi.expeca:sdr_tools:391 - waiting 1 times each 60 seconds for the sdr-09-tools to apply.
2023-09-01 09:17:45.703 | INFO     | chi.expeca:sdr_tools:412 - SERVICE=check_mango env variable is set
running check_mango...
You have chosen sdr-09 as a sta and speed check: True
SSHing to 10.30.1.17, username:root, password: root
running command on 10.30.1.17: 
/usr/sbin/ethtool wlan0 2>&1 | grep -q "Cannot get device settings: No such device" && echo "no" || echo "yes"
command stdout: 
yes
SSHing to 10.30.1.17, username:root, password: root
running command on 10.30.1.17: 
/usr/sbin/iw dev wlan0 info
command stdout: 
Interface wlan0
	ifindex 5
	wdev 0x2
	addr 40:d8:55:04:20:12
	ssid MANGO-AP
	type managed
	wiphy 0
	channel 1 (2412 MHz), width: 20 MHz, center1: 2412 MHz
	txpower 20.00 dBm
SSHing to 10.30.1.17, username:root, password: root
running command on 10.30.

## Configure Routing on SDRs

In [24]:
# configure routing on SDR-04 wifi access point using worker-10
sdrname = "sdr-04"
sdrnet = chi.network.get_network(sdrname+"-net")
sdr_tools(
    sdr_name = sdrname,
    sdr_net_id = sdrnet['id'],
    environment = {
        "SERVICE":"config_mango_routes",
        "SDR":sdrname,
        "SIDE":"ap",
        "JSON_PATH":"sdrs.json",
        "PROTOCOL":"udp",
        "SERVER_IP":"10.30.1.251",
        "SERVER_PORT":"2112",
        "AP_SERVER_PORT":"50500",
        "AP_STA_PORT":"50000",
        "STA_MAC_ADDR":"40:d8:55:04:20:19",
        "STA_IP":"192.168.11.3",
        "STA_AP_PORT":"50500"
    },
    waiting_iter = 1,
    waiting_sec = 10,
    key_str = "",
    verbose = False,
    worker_reservation_id = worker10_reservation_id,
    worker_net_interface = "ens1"
)

2023-09-01 10:15:50.510 | SUCCESS  | chi.expeca:sdr_tools:389 - created sdr-04-tools container.
2023-09-01 10:15:50.512 | INFO     | chi.expeca:sdr_tools:391 - waiting 1 times each 10 seconds for the sdr-04-tools to apply.
2023-09-01 10:16:02.052 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for sdr-04-tools container to be removed
2023-09-01 10:16:07.279 | INFO     | chi.expeca:wait_until_container_removed:37 - container sdr-04-tools is in Deleting state.
2023-09-01 10:16:12.503 | INFO     | chi.expeca:wait_until_container_removed:37 - container sdr-04-tools is in None state.


In [26]:
# configure routing on SDR-09 wifi station using worker-10
sdrname = "sdr-09"
sdrnet = chi.network.get_network(sdrname+"-net")
sdr_tools(
    sdr_name = sdrname,
    sdr_net_id = sdrnet['id'],
    environment = {
        "SERVICE":"config_mango_routes",
        "SDR":sdrname,
        "SIDE":"sta",
        "JSON_PATH":"sdrs.json",
        "PROTOCOL":"udp",
        "CLIENT_IP":"10.30.1.252",
        "CLIENT_PORT":"50000",
        "STA_CLIENT_PORT":"50000",
        "STA_AP_PORT":"50500",
        "AP_IP":"192.168.11.1",
        "AP_STA_PORT":"50000"
    },
    waiting_iter = 1,
    waiting_sec = 10,
    key_str = "",
    verbose = False,
    worker_reservation_id = worker10_reservation_id,
    worker_net_interface = "ens1"
)

2023-09-01 10:19:26.139 | SUCCESS  | chi.expeca:sdr_tools:389 - created sdr-09-tools container.
2023-09-01 10:19:26.140 | INFO     | chi.expeca:sdr_tools:391 - waiting 1 times each 10 seconds for the sdr-09-tools to apply.
2023-09-01 10:19:37.375 | INFO     | chi.expeca:wait_until_container_removed:30 - waiting 30 seconds for sdr-09-tools container to be removed
2023-09-01 10:19:42.609 | INFO     | chi.expeca:wait_until_container_removed:37 - container sdr-09-tools is in Deleting state.
2023-09-01 10:19:47.835 | INFO     | chi.expeca:wait_until_container_removed:37 - container sdr-09-tools is in None state.


# Configure the workload

## Server

In [27]:
sdrname = "sdr-04"
sdrnet = chi.network.get_network(sdrname+"-net")
chi.container.create_container(
    name = "edge-server",
    image = "samiemostafavi/perf-meas",
    reservation_id = worker10_reservation_id,
    environment = {},
    nets = [
        { "network" : sdrnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12409",
        "networks.1.ip":"10.30.1.251/24"
    },
)
chi.container.wait_for_active("edge-server")
logger.success("created edge-server container.")

2023-09-01 10:20:25.160 | SUCCESS  | __main__:<cell line: 17>:17 - created edge-server container.


## Client


In [28]:
sdrname = "sdr-09"
sdrnet = chi.network.get_network(sdrname+"-net")
chi.container.create_container(
    name = "end-node",
    image = "samiemostafavi/perf-meas",
    reservation_id = worker10_reservation_id,
    environment = {},
    nets = [
        { "network" : sdrnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12419",
        "networks.1.ip":"10.30.1.252/24"
    },
)
chi.container.wait_for_active("end-node")
logger.success("created end-node container.")

2023-09-01 10:20:58.792 | SUCCESS  | __main__:<cell line: 17>:17 - created end-node container.


# Run measurement command



```
irtt client -i 50ms -d 30s -l 172 --fill=rand 10.30.1.17:50000 --local=0.0.0.0:50000
```



# Tear Down

In [ ]:
# Remove everything and clean up

status = get_container_status("5g-nrue-01")
if status:
    chi.container.destroy_container("5g-nrue-01")
    wait_until_container_removed("5g-nrue-01")

status = get_container_status("5g-gnodeb-01")
if status:
    chi.container.destroy_container("5g-gnodeb-01")
    wait_until_container_removed("5g-gnodeb-01")

status = get_container_status("5gcn-7-spgwu")
if status:
    chi.container.destroy_container("5gcn-7-spgwu")
    wait_until_container_removed("5gcn-7-spgwu")

status = get_container_status("5gcn-6-smf")
if status:
    chi.container.destroy_container("5gcn-6-smf")
    wait_until_container_removed("5gcn-6-smf")

status = get_container_status("5gcn-5-amf")
if status:
    chi.container.destroy_container("5gcn-5-amf")
    wait_until_container_removed("5gcn-5-amf")

status = get_container_status("5gcn-4-ausf")
if status:
    chi.container.destroy_container("5gcn-4-ausf")
    wait_until_container_removed("5gcn-4-ausf")

status = get_container_status("5gcn-3-udm")
if status:
    chi.container.destroy_container("5gcn-3-udm")
    wait_until_container_removed("5gcn-3-udm")

status = get_container_status("5gcn-2-udr")
if status:
    chi.container.destroy_container("5gcn-2-udr")
    wait_until_container_removed("5gcn-2-udr")

status = get_container_status("5gcn-1-mysql")
if status:
    chi.container.destroy_container("5gcn-1-mysql")
    wait_until_container_removed("5gcn-1-mysql")

logger.info(f"stopped and removed all containers")

corenet = None
try:
    corenet = chi.network.get_network("5gcn-net")
except Exception as ex:
    logger.info(f"could not find 5gcn-net.")

if corenet:
    chi.network.delete_network(corenet["id"])
    logger.success(f"deleted the 5gcn-net")

# remove the leases
unreserve_byid(sdr02_lease["id"])
unreserve_byid(sdr03_lease["id"])
unreserve_byid(sdr08_lease["id"])
unreserve_byid(worker01_lease["id"])
unreserve_byid(worker02_lease["id"])
unreserve_byid(worker03_lease["id"])

2023-08-06 23:47:01.968 | INFO     | __main__:<cell line: 46>:46 - stopped and removed all containers
2023-08-06 23:47:09.064 | SUCCESS  | __main__:<cell line: 54>:56 - deleted the 5gcn-net
2023-08-06 23:47:09.912 | INFO     | chi.expeca:remove_lease:84 - Removing sdr-02-lease reservation with id 190074bf-d532-4ca4-8386-392a02f4359b.
2023-08-06 23:47:24.593 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b to become "None"
2023-08-06 23:47:29.869 | INFO     | chi.expeca:wait_until_lease_status:69 - lease sdr-02-lease with id 190074bf-d532-4ca4-8386-392a02f4359b is None.
2023-08-06 23:47:29.871 | SUCCESS  | chi.expeca:try_to_remove:95 - done
2023-08-06 23:47:30.267 | INFO     | chi.expeca:remove_lease:84 - Removing sdr-03-lease reservation with id 4793fee5-4a64-4fd6-9186-7e019d821ce3.
2023-08-06 23:47:44.323 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for sdr-03-lease with i